# Notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import neat
import matplotlib.pyplot as plt

import pickle
import multimodal_mazes
from tqdm import tqdm

path = '../Results/test10/'

## Initialisations

In [ ]:
# WIP: Initialisations 
path = '../'
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')

p = neat.Population(config)

In [ ]:
# WIP: Initialisations
genome = p.population[1]
node_names = {-1: 'Ch0 L', -2: 'Ch1 L', -3 : 'Ch0 R', -4 : 'Ch1 R', 
              -5: 'Ch0 U', -6: 'Ch1 U', -7 : 'Ch0 D', -8 : 'Ch1 D',
              0 : 'Act L', 1 : 'Act R', 2 : 'Act U', 3 : 'Act D', 4 : 'Wait'}
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

## Rule-based agents

### Fitness vs noise

In [ ]:
# Fitness vs noise
noises = np.linspace(start=0.0, stop=0.5, num=21)
policies = multimodal_mazes.AgentRuleBased.policies
colors = multimodal_mazes.AgentRuleBased.colors
results = np.zeros((len(noises), len(policies)))

# Generate mazes
maze = multimodal_mazes.TrackMaze(size=11, n_channels=2)
maze.generate(number=1000, noise_scale=0.0, gaps=0)

# Test agents
for a, noise in enumerate(tqdm(noises)):

    for b, policy in enumerate(policies): 
        agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], policy=policy)
        fitness = multimodal_mazes.eval_fitness(genome=None, config=None, channels=[1,1], sensor_noise_scale=noise, drop_connect_p=0.0, maze=maze, n_steps=6, agnt=agnt)

        results[a, b] = fitness

for b, policy in enumerate(policies): 
    plt.plot(noises, results[:,b], color=colors[b], label=policy)

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Sensor Noise')
plt.legend()

In [ ]:
# Maze viz
multimodal_mazes.plot_path(path=[], mz=maze.mazes[0], mz_goal_loc=maze.goal_locations[0], n_steps=6)
plt.ylim(3.5,6.5)

In [ ]:
# AUC
auc = np.trapz(y=results.T, x=noises)
for b, _ in enumerate(policies): 
    ml, sl, _ = plt.stem(b, auc[b])
    ml.set_color(colors[b])
    sl.set_color(colors[b])
plt.xticks(range(len(policies)), policies, rotation='vertical')
plt.ylabel('AUC');

### Fitness vs phi

In [ ]:
# Fitness vs phi parameter
from copy import deepcopy
from matplotlib import cm 
phis = np.linspace(start=0.0, stop=1.0, num=11)
policies = multimodal_mazes.AgentRuleBased.policies
results = np.zeros((len(phis), len(policies)))
cols = cm.get_cmap('plasma', len(policies))

# Generate mazes
maze = multimodal_mazes.GeneralMaze(size=11, n_channels=2)
maze.generate(number=1000, noise_scale=0.0)

# Calculate fitness
for a, phi in enumerate(tqdm(phis)):

    # Scale mazes 
    maze_scaled = deepcopy(maze)

    for n, _ in enumerate(maze_scaled.mazes):
        maze_scaled.mazes[n][:,:,0] *= (1 - phi) 
        maze_scaled.mazes[n][:,:,1] *= phi

    # Test agents
    for b, policy in enumerate(policies): 
        agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], policy=policy)
        fitness = multimodal_mazes.eval_fitness(genome=None, config=None, channels=[1,1], sensor_noise_scale=0.02, drop_connect_p=0.0, maze=maze_scaled, n_steps=50, agnt=agnt)

        results[a, b] = fitness

for b, policy in enumerate(policies): 
    plt.plot(phis, results[:,b], color=cols.colors[b], label=policy)

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Phi')
plt.legend()

In [ ]:
# WIP: Fitness vs phi AUC 
auc = np.trapz(y=results.T, x=phis)
for b, _ in enumerate(policies): 
    ml, sl, _ = plt.stem(b, auc[b])
    ml.set_color(cols.colors[b])
    sl.set_color(cols.colors[b])
plt.xticks(range(len(policies)), policies, rotation='vertical')
plt.ylabel('AUC');

### Fitting weights

In [ ]:
# Fitness vs phi parameter
from copy import deepcopy
from matplotlib import cm 
phis = np.linspace(start=0.0, stop=1.0, num=11)
policies = multimodal_mazes.AgentRuleBased.policies
results = np.zeros((len(phis), len(policies)))
cols = cm.get_cmap('plasma', len(policies))

# Generate mazes
maze = multimodal_mazes.GeneralMaze(size=11, n_channels=2)
maze.generate(number=1000, noise_scale=0.0)

# Calculate fitness
for a, phi in enumerate(tqdm(phis)):

    # Scale mazes 
    maze_scaled = deepcopy(maze)

    for n, _ in enumerate(maze_scaled.mazes):
        maze_scaled.mazes[n][:,:,0] *= (1 - phi) 
        maze_scaled.mazes[n][:,:,1] *= phi

    # Generate sensation-action pairs 
    maze_scaled.generate_sensation_action_pairs(sensor_noise_scale=0.02)

    # Test agents
    for b, policy in enumerate(policies): 
        agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], policy=policy)
        agnt.fit_channel_weights(n_weights=5, channel_inputs=maze_scaled.channel_inputs, ci_actions=maze_scaled.ci_actions)
        fitness = multimodal_mazes.eval_fitness(genome=None, config=None, channels=[1,1], sensor_noise_scale=0.02, drop_connect_p=0.0, maze=maze_scaled, n_steps=50, agnt=agnt)

        results[a, b] = fitness

for b, policy in enumerate(policies): 
    plt.plot(phis, results[:,b], color=cols.colors[b], label=policy)

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Phi')
plt.legend()

In [ ]:
# WIP: Fitness vs phi AUC 
auc = np.trapz(y=results.T, x=phis)
for b, _ in enumerate(policies): 
    ml, sl, _ = plt.stem(b, auc[b])
    ml.set_color(cols.colors[b])
    sl.set_color(cols.colors[b])
plt.xticks(range(len(policies)), policies, rotation='vertical')
plt.ylabel('AUC');

In [ ]:
# WIP: Example paths 
n = 0
path = multimodal_mazes.GeneralMaze.shortest_path(mz=maze.mazes[n], d_map=maze.d_maps[n], start=maze.start_locations[n], exit=maze.goal_locations[n])
# agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], memory_on=True)
# time, path = multimodal_mazes.maze_trial(mz=maze.mazes[n], mz_start_loc=maze.start_locations[n], mz_goal_loc=maze.goal_locations[n], channels=[1,1], sensor_noise_scale=0.0, drop_connect_p=0.0, n_steps=10, agnt=agnt)
multimodal_mazes.plot_path(path=path, mz=maze.mazes[n], mz_goal_loc=maze.goal_locations[n], n_steps=len(path))

### Memory

In [ ]:
# Fitness vs noise
noises = np.linspace(start=0.0, stop=0.5, num=11)
policies = multimodal_mazes.AgentRuleBasedMemory.policies
alphas = np.linspace(start=0.0, stop=2.0, num=11)
results = np.zeros((len(noises), len(policies), len(alphas)))

# Generate mazes
maze = multimodal_mazes.TrackMaze(size=11, n_channels=2)
maze.generate(number=1000, noise_scale=0.0, gaps=1)

# Test agents
for a, noise in enumerate(tqdm(noises)):

    for b, policy in enumerate(policies): 

        for c, alpha in enumerate(alphas):
            agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy=policy)
            agnt.alpha=alpha
            fitness = multimodal_mazes.eval_fitness(genome=None, config=None, channels=[1,1], sensor_noise_scale=noise, drop_connect_p=0.0, maze=maze, n_steps=6, agnt=agnt)
            results[a, b, c] = fitness

In [ ]:
# Fitness vs noise 
cols = ['#0ABAB5', 'xkcd:grey']
auc = np.trapz(y=results.T, x=noises, axis=2)
for b, policy in enumerate(policies): 
    plt.plot(alphas, auc[:,b], color=cols[b], label=policy)

plt.ylabel('AUC')
plt.xlabel(r"$\alpha$")
plt.legend()

## Evolved agents

### Multiple experiments

In [ ]:
# WIP: Comparing n experiments with m repeats 

# Building the feature matricies
import os
paths = ['../Results/test10/'] 
fitness_cutoff = 0.95

metrics_x, metrics_y, metrics_z = [], [], []
for a, path in enumerate(tqdm(paths)): 

    # Load config data 
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')
    exp_config = multimodal_mazes.load_exp_config(path + 'exp_config.ini')

    # Generate mazes
    maze = multimodal_mazes.TrackMaze(size=exp_config['maze_size'], n_channels=len(exp_config['channels']))
    # maze.generate(number=exp_config['n_mazes'], noise_scale=exp_config['maze_noise_scale'], gaps=exp_config['maze_gaps']) 
    maze.generate(number=exp_config['n_mazes'], noise_scale=exp_config['maze_noise_scale'], gaps=1) 

    # Load data 
    for f in os.listdir(path):
        if f.endswith(".npy"):
            
            print("Testing: " + str(a) + "-" + os.path.splitext(f)[0])
            exp_data = np.load(path + f)
            with open(path + os.path.splitext(f)[0] + '.pickle', 'rb') as file:
                genomes = pickle.load(file)

            # Select top agents 
            top_agents = multimodal_mazes.id_top_agents(fitness_cutoff=fitness_cutoff, exp_data=exp_data, maze=maze, exp_config=exp_config, genomes=genomes, config=config)

            if top_agents:
                # Calculate architecture metrics 
                top_agents_metrics_n, top_agents_metrics_p, mn_keys, mp_keys = multimodal_mazes.architecture_metrics_matrices(agents=top_agents, genomes=genomes, config=config)     
                results_x = np.concatenate((top_agents_metrics_n, top_agents_metrics_p), axis=1) 

                # Store data 
                metrics_x.append(results_x)
                metrics_y.append(np.ones(len(results_x)) * a)
                metrics_z.append(np.ones(len(results_x)) * int(os.path.splitext(f)[0]))

metrics_x = np.concatenate(metrics_x, axis=0) # agents x metrics 
metrics_y = np.concatenate(metrics_y, axis=0).astype(int) # agents, 
metrics_z = np.concatenate(metrics_z, axis=0) # agents, 
metrics_labels = list(mn_keys) + list(mp_keys) # metrics, 
metrics_type = np.concatenate((np.zeros(len(mn_keys)), np.ones(len(mp_keys)))) # metrics, 

assert len(metrics_x) == len(metrics_y) == len(metrics_z), "Mismatched data?"
assert metrics_x.shape[1] == len(metrics_labels) == len(metrics_type), "Mismatched labels?"

print(np.unique(metrics_y, return_counts=True))

In [ ]:
# WIP: focussing on unique solutions 
u, idx = np.unique(metrics_x, return_index=True, axis=0)
print(idx.shape)

In [ ]:
# WIP: Comparing features 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

model = DecisionTreeClassifier(class_weight="balanced")
scores = cross_val_score(estimator=model, X=metrics_x, y=metrics_y, cv=10)
print(scores)
model.fit(metrics_x, metrics_y)
plt.plot(model.feature_importances_) # sums to 1

In [ ]:
# WIP: Ranking features 
acc_per_metric = []
for f, _ in enumerate(metrics_labels): 
    model = DecisionTreeClassifier(class_weight="balanced", max_depth=1)
    scores = cross_val_score(estimator=model, X=metrics_x[:,f].reshape(-1,1), y=metrics_y, cv=10)
    acc_per_metric.append(scores.mean())

# WIP: Plot ranked features 
f, ax = plt.subplots(1, figsize=(10, 5))
metrics_sorted = np.argsort(acc_per_metric)[::-1]
plt.plot(np.array(acc_per_metric)[metrics_sorted], 'k')
v, c = np.unique(metrics_y, return_counts=True)
plt.hlines(y = max(c) / sum(c), xmin=0, xmax=len(metrics_sorted)-1, color="xkcd:gray", ls="dotted")
plt.xticks(range(len(metrics_labels)), [metrics_labels[l] for l in metrics_sorted], rotation='vertical')
plt.ylabel('Accuracy')

In [ ]:
# WIP: Plot just one feature
cols = ['xkcd:grey', 'k', 'k', '#0ABAB5']

best_feature = metrics_sorted[0]
for b in np.unique(metrics_y):
    parts = plt.violinplot(dataset=metrics_x[metrics_y==b,:][:, best_feature], positions=[b], showextrema=False, showmedians=True);
    for pc in parts['bodies']:
        pc.set_facecolor(cols[b])
        pc.set_edgecolor(cols[b])
        pc.set_alpha(0.5)

    vp = parts['cmedians']
    vp.set_edgecolor(cols[b])
    vp.set_alpha(1)

plt.ylabel(metrics_labels[best_feature])
# plt.xticks([0,1], ['A', 'B'])

In [ ]:
# WIP: All features plot  
cols = ['xkcd:grey', '#0ABAB5']
y_labels = ['Number', 'Value']
offsets = [-0.25, 0.25]

lines = []
f, ax = plt.subplots(1, 2, gridspec_kw={'width_ratios': [0.4, 0.6]}, figsize=(15, 5))
for a in [0,1]: # for each axis 
    plt.sca(ax[a])
    ms = metrics_sorted[metrics_type[metrics_sorted] == a]
    for b in [0,1]: # for each group 
        parts = plt.violinplot(dataset=metrics_x[metrics_y==b,:][:, ms], positions=np.linspace(start=0, stop=len(ms) - 1, num=len(ms))+offsets[b], showextrema=False, showmedians=True);
        for pc in parts['bodies']:
            pc.set_facecolor(cols[b])
            pc.set_edgecolor(cols[b])
            pc.set_alpha(0.5)

        vp = parts['cmedians']
        vp.set_edgecolor(cols[b])
        vp.set_alpha(1)

        if a == 0: lines.append(vp)

    ax[a].set_xticks(np.arange(len(ms)), [metrics_labels[i] for i in ms], rotation='vertical')
    ax[a].set_ylabel(y_labels[a])

# Legend 
plt.legend(lines, ['A, n=' + str(sum(metrics_y == 0)),
                   'B, n=' + str(sum(metrics_y == 1))])

### Single experiments

In [ ]:
# Load config data 
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')

exp_config = multimodal_mazes.load_exp_config(path + 'exp_config.ini')
print('Maze noise: ' + str(exp_config['maze_noise_scale']))
print('Sensor noise: ' + str(exp_config['sensor_noise_scale']))
print('Drop connect p: ' + str(exp_config['drop_connect_p']))

In [ ]:
# Load data 
x = np.load(path + '101.npy')

top_agent = np.where(x['fitness'] == x['fitness'].max())

with open(path + '101.pickle', 'rb') as file:
    genomes = pickle.load(file)

genome_id, genome, channels = genomes[top_agent[0][0]]

print(x[top_agent[0][0]])
print(genome.size())

In [ ]:
# Plotting

# Fitness 
multimodal_mazes.plot_fitness_over_generations(x, plot_species=True)

# Architecture
node_names = {-1: 'Ch0 L', -2: 'Ch1 L', -3 : 'Ch0 R', -4 : 'Ch1 R', 
              -5: 'Ch0 U', -6: 'Ch1 U', -7 : 'Ch0 D', -8 : 'Ch1 D',
              0 : 'Act L', 1 : 'Act R', 2 : 'Act U', 3 : 'Act D', 4 : 'Wait'}
genome = multimodal_mazes.prune_architecture(genome, config)
plt.figure()
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

In [ ]:
# Top genomes 
fitness_cutoff = 1.0

# Track maze 
maze = multimodal_mazes.TrackMaze(size=exp_config['maze_size'], n_channels=len(exp_config['channels']))
# maze.generate(number=exp_config['n_mazes'], noise_scale=exp_config['maze_noise_scale'], gaps=exp_config['maze_gaps']) 
maze.generate(number=exp_config['n_mazes'], noise_scale=exp_config['maze_noise_scale'], gaps=exp_config['maze_gaps']) 

# # General mazes 
# maze = multimodal_mazes.GeneralMaze(size=exp_config['maze_size'], n_channels=len(exp_config['channels']))
# maze.generate(number=exp_config['n_mazes'], noise_scale=exp_config['maze_noise_scale'])

# # Phi 
# phi = 0.8
# for n, _ in enumerate(maze.mazes):
#     maze.mazes[n][:,:,0] *= (1 - phi) 
#     maze.mazes[n][:,:,1] *= phi 

# Test agents 
top_agents = multimodal_mazes.id_top_agents(fitness_cutoff=fitness_cutoff, exp_data=x, maze=maze, exp_config=exp_config, genomes=genomes, config=config)
print(len(top_agents))

In [ ]:
# Visualising a path
n = 0
time, path = multimodal_mazes.maze_trial(mz=maze.mazes[n], mz_start_loc=maze.start_locations[n], mz_goal_loc=maze.goal_locations[n], channels=[1,1], sensor_noise_scale=0.02, drop_connect_p=0.0, n_steps=50, genome=genome, config=config)
multimodal_mazes.plot_path(path=path, mz=maze.mazes[n], mz_goal_loc=maze.goal_locations[n], n_steps=len(path))

In [ ]:
# Architecture metrics 
top_agents_metrics_n, top_agents_metrics_p, mn_keys, mp_keys = multimodal_mazes.architecture_metrics_matrices(agents=top_agents, genomes=genomes, config=config)

# Plotting 
f, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [0.4, 0.6]}, figsize=(15, 5))
a0.plot(top_agents_metrics_n.T, c='k', linewidth=1.5, alpha=0.25);
a0.set_xticks(np.arange(len(mn_keys)), mn_keys, rotation='vertical')
a0.set_ylabel('Number')

a1.plot(top_agents_metrics_p.T, c='k', linewidth=1.5, alpha=0.25);
a1.set_xticks(np.arange(len(mp_keys)), mp_keys, rotation='vertical')
a1.set_ylabel('Value')
a1.set_ylim([-0.025, 1.025])

In [ ]:
# WIP: Visualising networks with specific features 
# idx = np.where(top_agents_metrics_n[:,4] == top_agents_metrics_n[:,4].min()) # fewest edges
# idx = np.where(top_agents_metrics_n[:,4] == top_agents_metrics_n[:,4].max()) # most edges
# idx = np.where(top_agents_metrics_p[:,2] == top_agents_metrics_p[:,2].max()) # most ii connections
# idx = np.where(top_agents_metrics_p[:,-2] == top_agents_metrics_p[:,-2].min()) # lowest e:i ratio
# idx = np.where(top_agents_metrics_p[:,-3] == top_agents_metrics_p[:,-3].max()) # highest transitivity

# Mode
# v,c = np.unique(top_agents_metrics_p[:,2], return_counts=True)
# idx = np.where(top_agents_metrics_p[:,2] == v[np.argmax(c)]) 

_, genome, _ = genomes[top_agents[idx[0][0]]]
genome = multimodal_mazes.prune_architecture(genome, config)
plt.figure()
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

### Edit distances / costs

In [ ]:
# WIP: edit distances
edit_distances = []
for n in tqdm(top_agents):
    _, genome, _ = genomes[n]
    genome = multimodal_mazes.prune_architecture(genome, config)
    edit_distances.append(multimodal_mazes.edit_distance(genome, config))
plt.hist(edit_distances)

print(min(edit_distances))

In [ ]:
# Costs 
costs = []
for n in top_agents: 
    _, genome, _ = genomes[n]
    genome = multimodal_mazes.prune_architecture(genome, config)
    architecture_metrics_n, _ = multimodal_mazes.architecture_metrics(genome=genome, config=config, channels=[1,1])
    costs.append(architecture_metrics_n['$\\mathregular{\\eta}$'] + architecture_metrics_n['$\\mathregular{E}$'])

plt.hist(costs)

In [ ]:
# Lowest cost architecture
_, genome, _ = genomes[top_agents[np.argmin(costs)]]
genome = multimodal_mazes.prune_architecture(genome, config)
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

## Robustness

### Single experiments

In [ ]:
# Robustness to sensor noise
noise_scales = np.linspace(start=0.0, stop=0.5, num=11)
noise_results, noise_baseline = multimodal_mazes.robustness_to_sensor_noise(agents=top_agents, noise_scales=noise_scales, n_mazes=1000, exp_config=exp_config, genomes=genomes, config=config)
multimodal_mazes.plot_robustness(condition_values=noise_scales, condition_results=noise_results, condition_label='Sensor noise', noise_baseline=noise_baseline, agents=top_agents, genomes=genomes, config=config, node_names=node_names)
print(min(np.trapz(y=noise_results, x=noise_scales)), max(np.trapz(y=noise_results, x=noise_scales)))

In [ ]:
# Robustness to dropconnect 
drop_scales = np.linspace(start=0.0, stop=1.0, num=11)
drop_results, drop_baseline = multimodal_mazes.robustness_to_drop_connect(agents=top_agents, drop_connect_ps=drop_scales, n_mazes=100, exp_config=exp_config, genomes=genomes, config=config)
multimodal_mazes.plot_robustness(condition_values=drop_scales, condition_results=drop_results, condition_label='Drop connect p', noise_baseline=drop_baseline, agents=top_agents, genomes=genomes, config=config, node_names=node_names)
print(min(np.trapz(y=drop_results, x=drop_scales)), max(np.trapz(y=drop_results, x=drop_scales)))

## Multiple experiments - template architectures 

In [ ]:
# Define templates 
import networkx as nx
import networkx.algorithms.isomorphism as iso
em = iso.numerical_edge_match("weight", 1)

# Recurrent template 
edges = [[-1, 0], [-3, 1], [-2, 0], [-4, 1], [0,0], [1,1]] 
G_r = nx.DiGraph()
for e in edges:
    G_r.add_edge(e[0], e[1], weight=1)

# Hidden skip template 
edges = [[-1, 0], [-3, 1], [-2, 0], [-4, 1], [-1, 99], [-4,100], [99,0], [100,1]]
G_hs = nx.DiGraph()
for e in edges:
    G_hs.add_edge(e[0], e[1], weight=1)


In [ ]:
# Match genomes to templates
import os
paths = ['../Results/test10/'] 
fitness_cutoff = 1.0

costs, matching_genomes, mg_keys = [], [], []
for a, path in enumerate(tqdm(paths)): 

    # Load config data 
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')
    exp_config = multimodal_mazes.load_exp_config(path + 'exp_config.ini')

    # Generate mazes
    maze = multimodal_mazes.TrackMaze(size=exp_config['maze_size'], n_channels=len(exp_config['channels']))
    maze.generate(number=exp_config['n_mazes'], noise_scale=exp_config['maze_noise_scale'], gaps=1) 

    # Load data 
    for f in os.listdir(path):
        if f.endswith(".npy"):
            
            print("Testing: " + str(a) + "-" + os.path.splitext(f)[0])
            exp_data = np.load(path + f)
            with open(path + os.path.splitext(f)[0] + '.pickle', 'rb') as file:
                genomes = pickle.load(file)

            # Select top agents 
            top_agents = multimodal_mazes.id_top_agents(fitness_cutoff=fitness_cutoff, exp_data=exp_data, maze=maze, exp_config=exp_config, genomes=genomes, config=config)

            if top_agents:
                for n in top_agents:
                    
                    # Select genome 
                    _, genome, _ = genomes[n]
                    genome = multimodal_mazes.prune_architecture(genome, config)
                
                    # Define weighted graph
                    G, nodes, edges = multimodal_mazes.define_graph(genome=genome, weights='Binary')

                    # Costs 
                    costs.append(len(edges) + len(nodes))

                    # Isomorphism  
                    if (nx.is_isomorphic(G, G_r, edge_match=em)):
                        matching_genomes.append(genomes[n])
                        mg_keys.append(0)
                    elif (nx.is_isomorphic(G, G_hs, edge_match=em)):
                        matching_genomes.append(genomes[n])
                        mg_keys.append(1)

print(len(matching_genomes))
print(np.unique(mg_keys, return_counts=True))

In [ ]:
# Testing: plotting single networks 
node_names = {-1: 'Ch0 L', -2: 'Ch1 L', -3 : 'Ch0 R', -4 : 'Ch1 R', 
              -5: 'Ch0 U', -6: 'Ch1 U', -7 : 'Ch0 D', -8 : 'Ch1 D',
              0 : 'Act L', 1 : 'Act R', 2 : 'Act U', 3 : 'Act D', 4 : 'Wait'}

_, genome, _ = matching_genomes[0]
genome = multimodal_mazes.prune_architecture(genome, config)
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

In [ ]:
# Costs
cols = ['#0ABAB5', 'xkcd:grey']
labels = ['Recurrent outputs', 'Hidden skip']
 
plt.hist(costs, density=True, histtype='stepfilled', color='xkcd:grey', alpha=0.25, label='All solutions')
plt.axvline(x=len(G_r.nodes) + len(G_r.edges), color=cols[0], label=labels[0])
plt.axvline(x=len(G_hs.nodes) + len(G_hs.edges), color=cols[1], label=labels[1])

plt.ylabel('Density')
plt.xlabel('Cost\n($\mathregular{\eta + E}$)')
plt.legend()

In [ ]:
# Robustness to sensor noise
noises = np.linspace(start=0.0, stop=0.5, num=11)
noise_results, noise_baseline = multimodal_mazes.robustness_to_sensor_noise(agents=list(range(len(matching_genomes))), noise_scales=noises, n_mazes=1000, exp_config=exp_config, genomes=matching_genomes, config=config)

In [ ]:
# Plotting 
fig, ax = plt.subplots()
for a, mg_key in enumerate(np.unique(mg_keys)):
    plt.plot(noises, noise_results[mg_keys==mg_key,:].T, color=cols[a], alpha=0.5, label=labels[a]);
plt.plot(noises, noise_baseline.T, color='k', alpha=1.0, label='-/-');
plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Sensor Noise')
multimodal_mazes.unique_legend(ax=ax, order=[0,1,2], loc='upper right')

# Algorithm overlay 
plt.plot(noises, results[:,0,5], color=cols[0], alpha=1)
plt.plot(noises, results[:,1,3], color=cols[1], alpha=1)

## Exact baselines

In [ ]:
# # WIP: exact baseline 
# import itertools

# # Parameters 
# n_actions = [-1, 1, 0]
# n_time_steps = 10

# # Enumerate 
# n_action_sequences = len(n_actions) ** n_time_steps 
# n_paths = len(np.unique(n_actions)) ** n_time_steps 

# all_paths = list(itertools.product(np.unique(n_actions), repeat=n_time_steps))
# all_paths = np.array(all_paths, dtype='b')
# all_paths_cu = np.cumsum(all_paths, axis=1) 

# # Boundary conditions 
# all_paths_cu = np.clip(all_paths_cu, a_min=-4, a_max=4)

# fitness = []
# for path in all_paths_cu:
#     if min(path) == -4: # if you've reached the goal 
#         fitness.append(((1 - ((np.where(path == -4)[0][0] - 3) / (n_time_steps - 1 - 3))) + 1.0) * 0.5) # time + path 
#     else:
#         fitness.append((8 - (path[-1] - (-4))) / 8)

# print(np.mean(fitness))
# # exact_baseline = mean(fitness of each path * how often it occurs in the action sequences) 

# # Does this deal with left and right trials correctly? 